# Problem Nr. 2

> Now, you need to figure out how to pilot this thing.

## Part 1

The submarine can take sequences of commands in the syntax:

```
forward X
down X
up X
```

Our job is to calculare the final horizontal position and depth and supply the product of $horizontalPosition \times depth$

---- 

We'll start by preparing the environment and the input.

In [2]:
from IPython.display import display, Markdown,Latex
import pandas as pd

input_data = pd.read_csv('../../inputs/2/course-instructions.csv', sep=' ')


We'll need to handle our current position and how to change it when reading the commands.
Our initial position will be (0,0) (forward/backward,depth).

In [3]:
from typing import NamedTuple


class Position(NamedTuple):
    forward: int
    depth: int


current_position: Position = Position(forward=0, depth=0)

display(Markdown(f'The initial position is `{current_position}`.'))


The initial position is `Position(forward=0, depth=0)`.

Now, let's write some functions to handle the syntax of the input.

In [4]:
def handle_forward(magnitude: int, current_position: Position) -> Position:
    return current_position._replace(forward=current_position.forward +
                                     magnitude)


def handle_up(magnitude: int, current_position: Position) -> Position:
    return current_position._replace(depth=current_position.depth - magnitude)


def handle_down(magnitude: int, current_position: Position) -> Position:
    return current_position._replace(depth=current_position.depth + magnitude)


Now we still need to parse the output lines.

In [5]:
command_dict = {
    "forward": handle_forward,
    "up": handle_up,
    "down": handle_down
}


def handle_command(command: str, current_position: Position,
                   magnitude: int) -> Position:
    command_handler = command_dict[command]
    if command_handler is None:
        raise NameError(f'Encountered unknown command {command}.')
    else:
        return command_handler(magnitude=magnitude,
                               current_position=current_position)


And when done with this, everything comes together and we can start processing our commands.

In [6]:
for (index, direction, magnitude) in input_data.itertuples():
    current_position = handle_command(command=direction,current_position=current_position,magnitude=magnitude)

display(Markdown(f'The current_position after processing all commands is `{current_position}`.'))

The current_position after processing all commands is `Position(forward=2105, depth=807)`.

Now, it's trivial to provide the answer, which is the product of the forward position and depth.


In [7]:
answer_one = current_position.forward * current_position.depth
display(Markdown('------'))
display(Markdown(f'The product of the horizontal and vertical position is `{answer_one}`.'))

------

The product of the horizontal and vertical position is `1698735`.

Great! That was correct!

-----

## Part Two

> Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

We actually need to consider up and down to change our `aim` by `X` instead of our `depth`.
Then, forward will also change our aim by $X \times depth$.

Let's start by extending our Position class and giving it aim, then we can also change our command handlers.

In [8]:
class SubmarineStatus(NamedTuple):
    forward: int
    depth: int
    aim: int


current_status: SubmarineStatus = SubmarineStatus(forward=0, depth=0, aim=0)

Now all of our command handlers need to be able to deal with a `SubmarineStatus` instead of a `Position` and we also need to implement the aim logic.

In [9]:
def handle_forward_with_aim(
        magnitude: int, current_status: SubmarineStatus) -> SubmarineStatus:
    return current_status._replace(forward=current_status.forward + magnitude,
                                   depth=current_status.depth +
                                   current_status.aim * magnitude)


def handle_up_with_aim(magnitude: int,
                       current_status: SubmarineStatus) -> SubmarineStatus:
    return current_status._replace(aim=current_status.aim - magnitude)


def handle_down_with_aim(magnitude: int,
                         current_status: SubmarineStatus) -> SubmarineStatus:
    return current_status._replace(aim=current_status.aim + magnitude)


command_dict = {
    "forward": handle_forward_with_aim,
    "up": handle_up_with_aim,
    "down": handle_down_with_aim
}


def handle_command_with_aim(command: str, current_status: SubmarineStatus,
                            magnitude: int) -> SubmarineStatus:
    command_handler = command_dict[command]
    if command_handler is None:
        raise NameError(f'Encountered unknown command {command}.')
    else:
        return command_handler(magnitude=magnitude,
                               current_status=current_status)

The way in which we process the commands is pretty identical, but we'll need to call a slightly different function.

In [10]:
for (index, direction, magnitude) in input_data.itertuples():
    current_status = handle_command_with_aim(command=direction,
                                      current_status=current_status,
                                      magnitude=magnitude)

display(
    Markdown(
        f'The current_status after processing all commands is `{current_status}`.'
    ))


The current_status after processing all commands is `SubmarineStatus(forward=2105, depth=757618, aim=807)`.

In [11]:
answer_two = current_status.forward * current_status.depth
display(Markdown('------'))
display(Latex(f'The product of the horizontal and vertical position is: \({current_status.forward}\\times{current_status.depth}={answer_two}\).'))

------

<IPython.core.display.Latex object>

Hooray! Correct again!

-----